In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=0
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in
import argparse, subprocess
from importlib import reload
import cmasher as cmr

In [3]:
arr = pklload(f"./database/01_nh_ghmatch.pickle")
arr, arr.shape

(array([(   1, 1026,  6459716, 1,    1,    0, 0.85361391, 0.46676515, 0.47558907, 0.51981606,   38.10322621,   59.92555592,  42.18115311, 5.70733728e+10, 5.99459678e-04,  1278,  941453, 1, 1278,   0, 0.4667652 , 0.47558909, 0.51981645,   46.91326255,   60.74175161,  45.83347496, 1.17616941e+12, 0.00203744, 8.49744015e-06, 3.99073296e-07,  True,  True),
        (   2, 1026, 24790214, 1,    2,    0, 0.85361391, 0.50130222, 0.50063559, 0.5062117 ,   -2.49541847,   78.70240036, -48.00985953, 2.36677560e+11, 1.30097726e-03,  1417, 6741842, 1, 1417,   0, 0.50130211, 0.5006353 , 0.50621156,  -23.66507475,   81.6192084 , -38.93460233, 8.13017735e+12, 0.00388008, 0.00000000e+00, 3.38028877e-07,  True,  True),
        (   3, 1026,  2020156, 1,    3,    0, 0.85361391, 0.4753261 , 0.45961361, 0.47863688,  -19.2019251 ,   67.07041283,  29.44856101, 1.77113924e+10, 6.92913378e-04,  1664,  381110, 1, 1664,   0, 0.47532378, 0.45961278, 0.47863639,    4.02893669,   81.35941644,  35.13073288, 4.76106794

In [2]:
mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap1 = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snap1s = uri.TimeSeries(snap1)
snap1s.read_iout_avail()
nout1 = snap1s.iout_avail['iout']
gal1s = uhmi.HaloMaker.load(snap1, galaxy=True, path_in_repo='galaxy_old')

In [5]:
sats = gal1s[[269-1,  319-1,  368-1,  381-1,  410-1,  418-1, 419-1,  426-1,  456-1,  460-1,  467-1,
            481-1,  485-1,  488-1,  491-1,  495-1,  506-1,  509-1,  511-1,  513-1,  620-1,  621-1,
            623-1,  624-1,  625-1,  626-1,  627-1,  628-1,  633-1,  634-1,  639-1,  640-1,  641-1,
            645-1,  662-1,  664-1,  710-1,  711-1,  724-1,  725-1,  727-1,  791-1,  792-1,  830-1,
            831-1,  835-1,  836-1,  837-1,  840-1,  842-1,  854-1,  903-1,  968-1, 1006-1, 1008-1,
           1209-1]]
sats['id']

array([ 269,  319,  368,  381,  410,  418,  419,  426,  456,  460,  467,
        481,  485,  488,  491,  495,  506,  509,  511,  513,  620,  621,
        623,  624,  625,  626,  627,  628,  633,  634,  639,  640,  641,
        645,  662,  664,  710,  711,  724,  725,  727,  791,  792,  830,
        831,  835,  836,  837,  840,  842,  854,  903,  968, 1006, 1008,
       1209], dtype=int32)

In [ ]:
pouts = snap1s.iout_avail['iout'][snap1s.iout_avail['age'] >= snap1.age-0.5]
pouts = pouts[pouts < snap1.iout][::-1]
print(pouts)

rrange = 30*snap1.unit['kpc']


centers = {}
members = {}
all_scores = {}
give_scores = {}
take_scores = {}

def get_members(gal):
    global members, snap1s
    if(gal['timestep'] in members.keys()):
        if(gal['id'] in members[gal['timestep']].keys()):
            return members[gal['timestep']][gal['id']]
    else:
        members[gal['timestep']] = {}
    members[gal['timestep']][gal['id']] = uhmi.HaloMaker.read_member_part(snap1s.get_snap(gal['timestep']), gal['id'], galaxy=True, simple=True)
    return members[gal['timestep']][gal['id']]

for ip, pout in tqdm( enumerate(pouts), total=len(pouts) ):
    psnap = snap1s.get_snap(pout)
    # psnap.set_box(sat, 2*rrange)
    pgals = uhmi.HaloMaker.load(psnap, galaxy=True, double_precision=dp)
    for sat in sats:
        if(sat['id'] in centers.keys()):
            center = centers[sat['id']]
        else:
            center = [sat['x'], sat['y'], sat['z']]
            centers[sat['id']] = center
        my_member = get_members(sat)
        pneighbors = cut_box(pgals, *center, rrange)


        if(len(pneighbors)==0): continue

        give_score = np.zeros(len(pneighbors))
        take_score = np.zeros(len(pneighbors))
        for i, pg in enumerate(pneighbors):
            pmember = get_members(pg)
            intersect = np.sum( isin(pmember, my_member, assume_unique=True) )
            give_score[i] = intersect / len(my_member) / 2
            take_score[i] = intersect / len(pmember) / 2
        all_score = give_score * take_score
        
        argmax_all = np.argmax(all_score)
        argmax_give = np.argmax(give_score)
        argmax_take = np.argmax(take_score)
        if(not sat['id'] in all_scores.keys()):
            all_scores[sat['id']] = np.zeros(len(pouts))
            give_scores[sat['id']] = np.zeros(len(pouts))
            take_scores[sat['id']] = np.zeros(len(pouts))
        
        all_scores[sat['id']][ip] = pneighbors['id'][argmax_all] + all_score[argmax_all]
        # if(sat['id']==178): print(pneighbors['id'][argmax_all] + all_score[argmax_all], all_score[argmax_all])
        # if(sat['id']==178): print(give_score)
        give_scores[sat['id']][ip] = pneighbors['id'][argmax_give] + give_score[argmax_give]
        take_scores[sat['id']][ip] = pneighbors['id'][argmax_take] + take_score[argmax_take]
        centers[sat['id']] = [ pneighbors['x'][argmax_all], pneighbors['y'][argmax_all], pneighbors['z'][argmax_all] ]
            


